## Bibliotecas Utilizadas

In [29]:
import requests
from bs4 import BeautifulSoup
import json
import re
import numpy as np
import pandas as pd

## Extraindo Informações 1 - Medicamentos:

In [31]:
def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None


def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
    ('https://www.paguemenos.com.br/medicamentos-e-saude/alergias', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/antitabagismo', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/Aparelhos-para-saude', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/calmantes-naturais', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/Colesterol-e-trigliceridios', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/contraceptivos', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/cuidados-com-a-circulacao', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/dermatologicos', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/diabetes', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/disfuncao-eretil', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/Dor-Febre-e-Inflamacao', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/emagrecedores', 1),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/gastrointestinais-e-hepaticos', 50),
    ('https://www.paguemenos.com.br/generico', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/gripes-e-resfriados', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/oftalmico', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/ortopedicos', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/osteoporose', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/primeiros-socorros', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/reeducadores-intestinais', 50),
    ('https://www.paguemenos.com.br/medicamentos-e-saude/vitaminas-e-minerais', 50),
    ('https://www.paguemenos.com.br/medicamentos-especiais', 2),
    ('https://www.paguemenos.com.br/servicos-de-saude/vacinas', 1),
    ('https://www.paguemenos.com.br/servicos-de-saude/exames-de-analises-clinicas', 50),
    ('https://www.paguemenos.com.br/servicos-de-saude/servicos-basicos', 1)
]

informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)
df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_medicamentos.csv', index=False)



In [32]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-49538,Desloratadina 5mg Com 10 Comprimidos Generico ...,Desloratadina,49538,7896181922711,55.29,27.29,Desloratadina 5mg Com 10 Comprimidos Generico ...
1,sp-19382,Histamin 2mg Com 20 Comprimidos,Histamin,19382,7896714219301,16.63,9.69,<p><strong>HISTAMIN 2MG COM 20 COMPRIMIDOS</st...
2,sp-54596,Montelucaste Sodio 5mg Com 30 Comprimidos Gené...,Montelucaste,54596,7899095241725,69.20,33.49,Montelucaste Sodio 5mg Com 30 Comprimidos Gené...
3,sp-33111,Brometo Ipratropio Solução Para Inalação 20ml ...,Brometo,33111,7896004725420,17.75,5.19,Brometo Ipratropio Solução Para Inalação 20ml ...
4,sp-26539,"Maleato De Dexclofeniramina+betametasona 2+0,2...",Maleato De Dexclofeniramina,26539,7896004711485,22.68,10.39,"Maleato De Dexclofeniramina+betametasona 2+0,2..."
...,...,...,...,...,...,...,...,...
13382,sp-96549,Teste COVID+Influenza A/B,Clinic Farma,61873,7898968322028,59.90,59.90,"<h3>APRESENTAÇÃO</h3><p>O <a href=""https://www..."
13383,sp-94855,Aplicação De Brinco,Clinic Farma,61850,0000026185002,15.00,15.00,<h3>APRESENTAÇÃO</h3><p><strong>O Serviço de p...
13384,sp-60567,Bioimpedância,Clinic Farma,61859,0000026185941,15.75,15.00,Clinic Farma Teste De Bioimpedância
13385,sp-96552,Aferição de Pressão Arterial,Clinic Farma,61848,0000026184890,5.00,5.00,"<h3>APRESENTAÇÃO</h3><p>O <a href=""https://www..."


## Extraindo Informações 2 - Dermocosméticos:

In [33]:
def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

# Função principal
def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
   ('https://www.paguemenos.com.br/dermocosmeticos/cuidados-com-os-cabelos', 50),
    ('https://www.paguemenos.com.br/dermocosmeticos/cuidados-com-o-corpo', 50),
    ('https://www.paguemenos.com.br/dermocosmeticos/cuidados-com-o-rosto', 50),
    ('https://www.paguemenos.com.br/dermocosmeticos/protecao-solar', 50)
]

informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)
df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_dermocosmeticos.csv', index=False)



In [34]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-97443,Dercos Vichy Energy+ Shampoo Estimulante 400g,Vichy,74982,7899706254403,159.58,152.99,<h3>APRESENTAÇÃO</h3>\r\n\r\n<p>O Shampoo Derc...
1,sp-34974,Shampoo Celamina Ultra 150ml,Celamina,34974,7898928257087,77.99,77.99,"<h3 class=""ql-align-justify""><span style=""back..."
2,sp-102637,Minoxid.il Turbo 120ml,Miligrama,None,7908650111769,0.00,0.00,<p>A **Solução Capilar Minoxidil Turbinado** é...
3,sp-102673,Pill Food 120 Cápsulas,Miligrama,None,7908650107922,0.00,0.00,<p>**Pill Food** é um suplemento que combina v...
4,sp-55347,Pilexil Shampo Anticaspa Seca 150ml,Pilexil,55347,8430340038395,95.99,76.79,Pilexil Shampo Anticaspa Seca 150ml
...,...,...,...,...,...,...,...,...
2004,sp-54805,NIVEA SUN Protetor Solar Beauty Expert Corpo F...,Nivea,54805,4005900696212,55.64,52.99,<h3>APRESENTAÇÃO:</h3><p>A Nivea é reconhecida...
2005,sp-52533,Protetor Solar Facil Isdin Fusion Water Fps50 ...,Isdin,52533,8429420154940,99.90,99.90,Protetor Solar Facil Isdin Fusion Water Fps50 ...
2006,sp-51088,Protetor Solar Episol Color Extra Clara Fps70 40g,Episol,51088,7891142204028,96.99,96.99,Protetor Solar Episol Color Extra Clara Fps70 40g
2007,sp-43657,Protetor Solar Episol Color Clara Fps70 40g,Episol,43657,7891142145680,78.99,78.99,Episol Color é uma linha de proteção solar com...


## Extraindo Informações 3 - Mamãe e Bebê:

In [35]:
def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

# Função principal
def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
    ('https://www.paguemenos.com.br/mamaes-e-bebes/acessorios', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/alimentos', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/banho', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/fraldas', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/higiene-bucal', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/protecao-solar', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/repelentes', 50),
    ('https://www.paguemenos.com.br/mamaes-e-bebes/roupas', 50),  
    ('https://www.paguemenos.com.br/mamaes-e-bebes/troca-de-fralda', 50)
]

informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)
df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_mamãe_e_bebê.csv', index=False)


In [36]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-55714,Aspirador Nasal Likluc Aspirar Baby,Likluc,55714,0751320866066,104.99,99.99,<h3>APRESENTAÇÃO:</h3><p>O Aspirador nasal Lik...
1,sp-92523,Suganon 5mg Com 30 Comprimidos,Suganon,62496,7891317005641,152.69,131.99,Suganon 5mg Com 30 Comprimidos
2,sp-51491,Chupeta Avent Recém Nascido De 0 A 2 Meses,Philips Avent,51491,8710103776666,30.44,28.99,Chupeta Avent Recém Nascido De 0 A 2 Meses
3,sp-115117,Sugador Nasal - Sana Babies,Sana Babies,None,7898952193023,0.00,0.00,ASPIRADOR NASAL - SANA BABIESO Sugador Nasal d...
4,sp-115063,Mamatutti Relactação e Suplementação Alimentar...,Tutti Care,None,7898342430394,0.00,0.00,"Facilita a amamentação.Relactação, ou translac..."
...,...,...,...,...,...,...,...,...
2441,sp-50253,Pomada para Prevenção de Assaduras Dermodex Pr...,Dermodex,50253,7891035090066,12.99,12.99,<strong>APRESENTAÇÃO:</strong><br>\r\nDermodex...
2442,sp-46326,Lenços Umedecidos Johnsons Baby Toque Fresquin...,Johnsons Baby,46326,7891010657529,13.43,12.79,Os Lenços Umedecidos Johnson's Baby Toque Fres...
2443,sp-45051,Lenços Umedecidos Pompom Suave Leve 100 Pague ...,Pompom,45051,7896012878019,19.94,18.99,Lenços Umedecidos Pompom Suave Leve 100 Pague ...
2444,sp-44024,Lenços Umedecidos Pampers Sensitive Com 168 Un...,Pampers,44024,4015400636434,49.34,46.99,<h3>APRESENTAÇÃO:</h3><p>Os lenços umedecidos ...


## Extraindo Informações 4 - Cuidados Pessoais e Beleza:

In [37]:
def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
   ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/absorventes', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/acessorios', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/banho', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/cuidados-com-os-cabelos', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/cuidados-com-o-corpo', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/cuidados-com-os-pes-e-mao', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/cuidados-com-o-rosto', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/cuidados-intimos', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/descolorantes', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/desodorantes', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/geriatrico', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/higiene-bucal', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/homem', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/maquiagem', 50),
    ('https://www.paguemenos.com.br/cuidados-pessoais-e-beleza/perfumes-e-colonias', 50)
]

informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)
df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_cuidados_pessoais_e_beleza.csv', index=False)




In [38]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-43440,Absorvente Noturno Always Noites Tranquilas Su...,Always,43440,7506339326055,29.49,29.49,<h3>APRESENTAÇÃO:</h3><p>Apresentamos o Absorv...
1,sp-44015,Absorvente Noturno Always Noites Tranquilas Se...,Always,44015,7506309805498,27.49,27.49,"<h3 class=""ql-align-justify""><span style=""colo..."
2,sp-43434,Absorvente Noturno Always Noites Tranquilas Se...,Always,43434,7506339325263,29.49,29.49,"<h3 class=""ql-align-justify""><span style=""colo..."
3,sp-50911,Absorvente Always Super Proteção Suave Com Aba...,Always,50911,7500435126199,16.99,16.99,"<h3 class=""ql-align-justify""><span style=""colo..."
4,sp-45304,Absorvente Noturno Always Noites Tranquilas Su...,Always,45304,7506339394603,8.39,8.39,<h3>APRESENTAÇÃO:</h3><p>Descubra o Absorvente...
...,...,...,...,...,...,...,...,...
12233,sp-122610,Chic De Carolina Herrera Eau De Toilette Mascu...,Carolina Herrera,None,8411061311608,0.00,0.00,<p><strong>Chic For Men</strong> de Carolina H...
12234,sp-120952,Giorgio Armani Sì Eau de Parfum Intense - Perf...,Giorgio Armani,None,3614273734882,0.00,0.00,Giorgio Armani Si Eau De Parfum Intense - Perf...
12235,sp-121125,1 Million De Paco Rabanne Eau De Toilette Masc...,Paco Rabanne,None,3349668566372,0.00,0.00,"<p>O perfume <strong>1 Million</strong>, da re..."
12236,sp-120789,Jean Paul Gaultier Classique Parfum 100 ml,Jean Paul Gaultier,None,8435415011341,0.00,0.00,<p><strong>Jean Paul Gaultier Classique Parfum...


## Extraindo Informações 5 - Fitness e Nutrição:

In [41]:
def extrai_json(response):
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        json_txt = soup.find('template', {'data-varname': '__STATE__'}).find('script').text
        return json.loads(json_txt)
    except Exception as e:
        print(f"Erro ao extrair JSON: {e}")
        return None

def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
    ('https://www.paguemenos.com.br/fitness-e-nutricao/acessorios', 20),
    ('https://www.paguemenos.com.br/fitness-e-nutricao/cereais-e-fibras', 20),
    ('https://www.paguemenos.com.br/fitness-e-nutricao/diet', 20),
    ('https://www.paguemenos.com.br/fitness-e-nutricao/energeticos-e-isotonicos', 20),
    ('https://www.paguemenos.com.br/fitness-e-nutricao/suplementos', 20),
    ('https://www.paguemenos.com.br/fitness-e-nutricao/emagrecedores', 20)
]


informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)


df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_fitness_e_nutricao.csv', index=False)


In [43]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-112642,Faixa Elástica para Exercícios Mercur Amarela ...,Mercur,None,7896342907106,0.0,0.0,@import url(https://cdnjs.cloudflare.com/ajax/...
1,sp-112625,Faixa Elástica para Exercícios Mercur Vermelho...,Mercur,None,7896342907175,0.0,0.0,@import url(https://cdnjs.cloudflare.com/ajax/...
2,sp-112885,Faixa Elástica para Exercícios TheraBand Verde...,Chantal,None,7898493366023,0.0,0.0,.description-prod { color: #000000; ...
3,sp-112702,Faixa Elástica para Exercícios TheraBand Azul ...,Chantal,None,7898493366030,0.0,0.0,.description-prod { color: #000000; ...
4,sp-118611,Mini Pistola Massageadora Dellamed Bivolt,Dellamed,None,7908607305746,0.0,0.0,Mini Pistola Massageadora Dellamed Experiment...
...,...,...,...,...,...,...,...,...
1313,sp-103469,Insea 2 250mg 120 Cápsulas,Miligrama,None,7908650105683,0.0,0.0,<p>**InSea 2 - Bloqueador de Carboidratos (250...
1314,sp-102719,Slendesta 300mg 120 Cápsulas,Miligrama,None,7908650109339,0.0,0.0,<p>**Slendesta 300mg - 120 Cápsulas**</p><p><b...
1315,sp-102724,ID-alG 200mg 120 Cápsulas,Miligrama,None,7908650105577,0.0,0.0,<p>**ID-aIG 200mg - 120 Cápsulas**</p><p><br><...
1316,sp-102746,"ID-aLG 200mg + Saffrin 176,5mg 60 Cápsulas",Miligrama,None,7908650105560,0.0,0.0,"<p>**ID-aLG 200mg + Saffrin 176,5mg - 60 Cápsu..."


## Extraindo Informações 6 - Manipulação:

In [44]:
def extrai_json(response):
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        json_txt = soup.find('template', {'data-varname': '__STATE__'}).find('script').text
        return json.loads(json_txt)
    except Exception as e:
        print(f"Erro ao extrair JSON: {e}")
        return None

def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
    ('https://www.paguemenos.com.br/manipulacao/cuidados-com-a-pele', 50),
    ('https://www.paguemenos.com.br/manipulacao/saude-e-qualidade-de-vida', 50),
    ('https://www.paguemenos.com.br/manipulacao/emagrecedores', 50),
    ('https://www.paguemenos.com.br/manipulacao/cabelos-e-unhas', 50),
    ('https://www.paguemenos.com.br/manipulacao/pre-e-pos-treino', 50)
]


informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)


df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_Manipulação.csv', index=False)


In [45]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-105713,Tratamento Micose de Unha Uso Tópico - Óleo de...,Miligrama,None,7908650110069,0.0,0.0,Benefícios- Ação antifúngica tópica para o tra...
1,sp-106665,Trio Clareador 30 Doses,Miligrama,None,7908650110151,0.0,0.0,Benefícios:- Auxilia no aumento do colágeno e ...
2,sp-106628,Pill Food Maxx Turbinado com Silicio Orgânico ...,Miligrama,None,7908650107953,0.0,0.0,Benefícios:- Ajuda no fortalecimento dos fios-...
3,sp-105666,Pantocare 180 Cápsulas,Miligrama,None,7908650107731,0.0,0.0,Benefícios:- Ajuda a reduzir a perda de cabelo...
4,sp-105742,Loção Clareadora para Axilas e Virilhas Roll-o...,Miligrama,None,7908650106260,0.0,0.0,Benefícios:- Promove o clareamento das manchas...
...,...,...,...,...,...,...,...,...
469,sp-102718,L-Glutamina 300g,Miligrama,None,7908650106055,0.0,0.0,<p>**L-Glutamina 300g**</p><p><br></p><p>**Ben...
470,sp-106700,Pre-Workout 30 Sachês Sabor Laranja,Miligrama,None,7908650108271,0.0,0.0,Benefícios:- Promover o aumento da resistência...
471,sp-93040,Pré-treino Força e Resistência – 30 Sachês (Sa...,Manipulação Pague Menos,None,,0.0,0.0,strong>Composição:</strong><br><br>\r\nBeta al...
472,sp-102771,Cafeína 100mg 180 Cápsulas,Miligrama,None,7908650101425,0.0,0.0,<p>A Cafeína é um composto químico amplamente ...


## Extraindo Informações 7 - Conveniência:

In [46]:
def extrai_json(response):
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        json_txt = soup.find('template', {'data-varname': '__STATE__'}).find('script').text
        return json.loads(json_txt)
    except Exception as e:
        print(f"Erro ao extrair JSON: {e}")
        return None

def get_price_without_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['ListPrice']
    except KeyError:
        return None

def get_price_with_discount(json_data, id):
    try:
        return json_data[f'$Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0.sellers.0.commertialOffer']['Price']
    except KeyError:
        return None

def get_ean(json_data, id):
    try:
        return json_data[f'Product:{id}.items({{"filter":"ALL_AVAILABLE"}}).0']['ean']
    except KeyError:
        return None

def get_last_description(json_data, id):
    try:
        descriptions = [valor['description'] for chave, valor in json_data.items() if chave.startswith(f'Product:{id}') and 'description' in valor]
        return descriptions[-1] if descriptions else None
    except KeyError:
        return None

def extrair_jsons_e_informacoes_produtos(categorias):
    produtos = []
    for categoria, num_paginas in categorias:
        for i in range(1, num_paginas+1):
            url = f'{categoria}?page={i}'
            try:
                response = requests.get(url)
                json_data = extrai_json(response)
                if json_data:
                    for chave, valor in json_data.items():
                        if chave.startswith('Product:'):
                            produto_id = chave.split(':')[-1]
                            if 'productName' in valor:
                                produto_nome = valor['productName']
                                marca = valor.get('brand', 'Marca não encontrada')
                                sku = valor.get('productReference', 'SKU não encontrado')
                                ean = get_ean(json_data, produto_id)
                                preco_sem_desconto = get_price_without_discount(json_data, produto_id)
                                preco_com_desconto = get_price_with_discount(json_data, produto_id)
                                descricao = get_last_description(json_data, produto_id)

                                produtos.append({
                                    'Produto ID': produto_id, 
                                    'Nome': produto_nome, 
                                    'Marca': marca, 
                                    'SKU': sku, 
                                    'EAN': ean, 
                                    'Preço sem Desconto': preco_sem_desconto, 
                                    'Preço com Desconto': preco_com_desconto,
                                    'Descrição': descricao
                                })
            except requests.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
    return produtos


categorias = [
    ('https://www.paguemenos.com.br/conveniencia/alimentos-e-bebidas', 50),
    ('https://www.paguemenos.com.br/brinquedos', 1),
    ('https://www.paguemenos.com.br/conveniencia/roupas-e-acessorios', 50),
    ('https://www.paguemenos.com.br/conveniencia/utilidades-e-outros', 50)
]


informacoes_produtos = extrair_jsons_e_informacoes_produtos(categorias)


df = pd.DataFrame(informacoes_produtos)
df = df.drop_duplicates()
df.to_csv('informacoes_produtos_Conveniencia.csv', index=False)

In [47]:
df

,Produto ID,Nome,Marca,SKU,EAN,Preço sem Desconto,Preço com Desconto,Descrição
0,sp-116676,Sorvete Kibon Cornetto spotify choco brownie,Cornetto,73414,7891150087484,12.60,12.00,Sorvete Kibon Cornetto spotify choco brownie
1,sp-46239,Picolé Magnum Cookies & Cream,Magnum,46239,7891150041042,15.00,15.00,Picolé Magnum Cookies & Cream
2,sp-17873,Bombom Gelado Eskibom Baun Mini,Eskibon,17873,7891075081215,13.29,13.29,<h3>APRESENTAÇÃO:</h3><p>O Mini Eskibon Kibon ...
3,sp-60564,Picole Magnum Chocolate Zero Açúcar,Kibon,60089,7891150081963,15.00,15.00,Picole Magnum Chocolate Zero Açúcar
4,sp-102259,Sorvete Corneto M&Ms,Kibon,54808,7891150065925,12.29,12.29,Sorvete Corneto M&Ms
...,...,...,...,...,...,...,...,...
2907,sp-118205,Desodorante Wild musk Rollon 70ml,Wild Musk,2775,7891166411334,0.00,0.00,Desodorante Wild musk Rollon 70ml
2908,sp-118207,Mamadeira Fiona C Desenho 60cc 7226,Fiona,3028,7896033272261,0.00,0.00,Mamadeira Fiona C Desenho 60cc 7226
2909,sp-118173,Mamadeira Super Tutti Frutti 240c 9231,Outras Marcas,587,7896033292313,0.00,0.00,Mamadeira Super Tutti Frutti 240c 9231
2910,sp-118208,Bronzeador delial Fps16 200ml,Delial,3078,7891106901116,11.49,11.49,<h3>APRESENTAÇÃO:</h3> <p>O Bronzeador Delial ...
